In [1]:
# check what directory I am in
# should be "code" directory in my blog draft post
import os
from dotenv import load_dotenv

load_dotenv('.env.public2', override=True)


True

In [2]:
connection_string = os.getenv('CONN_STRING')

In [3]:
print(connection_string)

Driver={ODBC Driver 18 for SQL Server};Server=tcp:sqlservercentralpublic.database.windows.net,1433;Database=AdventureWorks;Uid=sqlfamily;Pwd=sqlf@m1ly;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;


In [4]:
import pyodbc
conn = pyodbc.connect(connection_string)
print(conn)

In [6]:
statement = """
SELECT DISTINCT
  TABLE_SCHEMA
FROM INFORMATION_SCHEMA.TABLES
ORDER BY TABLE_SCHEMA
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('SalesLT',), ('sys',)]


In [7]:
statement = """
SELECT 
  *
FROM INFORMATION_SCHEMA.TABLES
ORDER BY TABLE_SCHEMA
"""
cursor = conn.cursor()
cursor.execute(statement)
print(cursor.fetchall())
cursor.close()

[('AdventureWorks', 'SalesLT', 'Logger', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'Customer', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'ProductModel', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'ProductDescription', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'Product', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'ProductModelProductDescription', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'ProductCategory', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'Address', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'CustomerAddress', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'SalesOrderDetail', 'BASE TABLE'), ('AdventureWorks', 'SalesLT', 'SalesOrderHeader', 'BASE TABLE'), ('AdventureWorks', 'sys', 'database_firewall_rules', 'VIEW')]


In [8]:
from tabulate import tabulate

statement = """
SELECT 
  TABLE_NAME, TABLE_TYPE
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'SalesLT'
ORDER BY TABLE_NAME
"""

cursor = conn.cursor()
cursor.execute(statement)

headers = [h[0] for h in cursor.description]
tables = cursor.fetchall()
print(tabulate(tables, headers=headers))

cursor.close()

TABLE_NAME                      TABLE_TYPE
------------------------------  ------------
Address                         BASE TABLE
Customer                        BASE TABLE
CustomerAddress                 BASE TABLE
Logger                          BASE TABLE
Product                         BASE TABLE
ProductCategory                 BASE TABLE
ProductDescription              BASE TABLE
ProductModel                    BASE TABLE
ProductModelProductDescription  BASE TABLE
SalesOrderDetail                BASE TABLE
SalesOrderHeader                BASE TABLE


In [47]:
def table_info(table, schema='SalesLT'):
    
    statement = (
        f"SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH\n"
        f"FROM INFORMATION_SCHEMA.COLUMNS\n"
        f"WHERE TABLE_SCHEMA = '{str(schema)}'\n"
        f"AND TABLE_NAME = '{str(table)}'"
    )

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        table_rows = cursor.fetchall()
        
    return table_rows, headers

In [69]:
table, head = table_info('vProductModelCatalogDescription')
print(tabulate(table, headers=head))

COLUMN_NAME             DATA_TYPE           CHARACTER_MAXIMUM_LENGTH
----------------------  ----------------  --------------------------
ProductModelID          int
Name                    nvarchar                                  50
Summary                 nvarchar                                  -1
Manufacturer            nvarchar                                  -1
Copyright               nvarchar                                  30
ProductURL              nvarchar                                 256
WarrantyPeriod          nvarchar                                 256
WarrantyDescription     nvarchar                                 256
NoOfYears               nvarchar                                 256
MaintenanceDescription  nvarchar                                 256
Wheel                   nvarchar                                 256
Saddle                  nvarchar                                 256
Pedal                   nvarchar                                 256
BikeFr

In [65]:
def constraint_info(table, schema='SalesLT'):

    statement = (
        f"SELECT TABLE_NAME, CONSTRAINT_NAME, CONSTRAINT_TYPE\n"
        f"FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS\n"
        f"WHERE TABLE_SCHEMA = '{str(schema)}'\n"
        f"AND TABLE_NAME = '{str(table)}'"
    )

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        constraint_rows = cursor.fetchall()
        
    return constraint_rows, headers

In [70]:
table, head = constraint_info('vProductModelCatalogDescription')
print(table)

[]


In [148]:
cursor = conn.cursor()

schema_set = set()

for row in cursor.tables():
    schema_set.add(row.table_schem)
    
print('Schema Name')
print('-' * 11)
print(*schema_set, sep='\n')

cursor.close()


Schema Name
-----------
SalesLT
dbo
INFORMATION_SCHEMA
sys


In [45]:
cursor = conn.cursor()

print(dir(cursor)

cursor.close()

In [88]:
def view_def(view, schema='SalesLT'):

    statement = (
    f"SELECT VIEW_DEFINITION\n"
    f"FROM INFORMATION_SCHEMA.VIEWS\n"
    f"WHERE TABLE_SCHEMA = '{str(schema)}'\n"
    f"AND TABLE_NAME = '{str(view)}'"
    )

    view_definition = str()
    with conn.cursor() as cursor:
        cursor.execute(statement)
        for row in cursor.fetchone():
            view_definition = view_definition + row + '\n'
       
    return view_definition

In [89]:
print(view_def('vProductAndDescription'))

CREATE VIEW [SalesLT].[vProductAndDescription]
WITH SCHEMABINDING
AS
-- View (indexed or standard) to display products and product descriptions by language.
SELECT
    p.[ProductID]
    ,p.[Name]
    ,pm.[Name] AS [ProductModel]
    ,pmx.[Culture]
    ,pd.[Description]
FROM [SalesLT].[Product] p
    INNER JOIN [SalesLT].[ProductModel] pm
    ON p.[ProductModelID] = pm.[ProductModelID]
    INNER JOIN [SalesLT].[ProductModelProductDescription] pmx
    ON pm.[ProductModelID] = pmx.[ProductModelID]
    INNER JOIN [SalesLT].[ProductDescription] pd
    ON pmx.[ProductDescriptionID] = pd.[ProductDescriptionID];




In [204]:
statement = """
SELECT
  TABLE_NAME, TABLE_TYPE
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'SalesLT'
ORDER BY TABLE_NAME
"""
cursor = conn.cursor()
cursor.execute(statement)

columns = [column[0] for column in cursor.description]
print(*columns, sep=', ')

for row in cursor:
    print(*row, sep=',  ')

cursor.close()

TABLE_NAME, TABLE_TYPE
Address,  BASE TABLE
Customer,  BASE TABLE
CustomerAddress,  BASE TABLE
Product,  BASE TABLE
ProductCategory,  BASE TABLE
ProductDescription,  BASE TABLE
ProductModel,  BASE TABLE
ProductModelProductDescription,  BASE TABLE
SalesOrderDetail,  BASE TABLE
SalesOrderHeader,  BASE TABLE
vGetAllCategories,  VIEW
vProductAndDescription,  VIEW
vProductModelCatalogDescription,  VIEW


In [16]:
cursor = conn.cursor()

for row in cursor.tables(schema='SalesLT'):
    print(*row, sep=', ')

# Can only get columns after I get data in the cursor
columns = [column[0] for column in cursor.description]
print(*columns, sep=', ')

cursor.close()

AdventureWorks, SalesLT, Address, TABLE, None
AdventureWorks, SalesLT, Customer, TABLE, None
AdventureWorks, SalesLT, CustomerAddress, TABLE, None
AdventureWorks, SalesLT, Logger, TABLE, None
AdventureWorks, SalesLT, Product, TABLE, None
AdventureWorks, SalesLT, ProductCategory, TABLE, None
AdventureWorks, SalesLT, ProductDescription, TABLE, None
AdventureWorks, SalesLT, ProductModel, TABLE, None
AdventureWorks, SalesLT, ProductModelProductDescription, TABLE, None
AdventureWorks, SalesLT, SalesOrderDetail, TABLE, None
AdventureWorks, SalesLT, SalesOrderHeader, TABLE, None
table_cat, table_schem, table_name, table_type, remarks


In [17]:
cursor = conn.cursor()

column_list = (
    cursor.columns(
        table='Address', 
        schema='SalesLT')
    .fetchall()
)

# headers
headers = [h[0] for h in cursor.description]
print(f'{headers[3]:{25}}{headers[5]:{20}}{headers[6]:{11}}')

# data
for row in column_list:
    print(f'{row[3]:{25}}{row[5]:{20}}{row[6]:{6}}')

cursor.close()

column_name              type_name           column_size
AddressID                int identity            10
AddressLine1             nvarchar                60
AddressLine2             nvarchar                60
City                     nvarchar                30
PostalCode               nvarchar                15
rowguid                  uniqueidentifier        36
ModifiedDate             datetime                23


In [18]:
cursor = conn.cursor()

table = 'Address'
schema = 'SalesLT'

primary_keys = cursor.primaryKeys(table=table, schema=schema).fetchall()
print(f"{table}:  Primary Keys = {primary_keys}")

foreign_keys = cursor.foreignKeys(table=table, schema=schema).fetchall()
print(f"{table}:  Foreign Keys = {foreign_keys}")

cursor.close()

Address:  Primary Keys = [('AdventureWorks', 'SalesLT', 'Address', 'AddressID', 1, 'PK_Address_AddressID')]
Address:  Foreign Keys = [('AdventureWorks', 'SalesLT', 'Address', 'AddressID', 'AdventureWorks', 'SalesLT', 'CustomerAddress', 'AddressID', 1, 1, 1, 'FK_CustomerAddress_Address_AddressID', 'PK_Address_AddressID', 7), ('AdventureWorks', 'SalesLT', 'Address', 'AddressID', 'AdventureWorks', 'SalesLT', 'SalesOrderHeader', 'BillToAddressID', 1, 1, 1, 'FK_SalesOrderHeader_Address_BillTo_AddressID', 'PK_Address_AddressID', 7), ('AdventureWorks', 'SalesLT', 'Address', 'AddressID', 'AdventureWorks', 'SalesLT', 'SalesOrderHeader', 'ShipToAddressID', 1, 1, 1, 'FK_SalesOrderHeader_Address_ShipTo_AddressID', 'PK_Address_AddressID', 7)]


In [19]:
cursor = conn.cursor()

column_list = (
    cursor.columns(
        table='vProductAndDescription', 
        schema='SalesLT')
    .fetchall()
)

# headers
headers = [h[0] for h in cursor.description]
print(f'{headers[3]:{25}}{headers[5]:{20}}{headers[6]:{11}}')

# data
for row in column_list:
    print(f'{row[3]:{25}}{row[5]:{20}}{row[6]:{6}}')

cursor.close()

column_name              type_name           column_size


In [100]:
from tabulate import tabulate

with conn.cursor() as cursor:
    tables =  cursor.tables(schema='SalesLT').fetchall()
    headers = [h[0] for h in cursor.description]

print(tabulate(tables, headers='keys'))




0                  1        2                                3      4
-----------------  -------  -------------------------------  -----  ---
data-science-test  SalesLT  Address                          TABLE
data-science-test  SalesLT  Customer                         TABLE
data-science-test  SalesLT  CustomerAddress                  TABLE
data-science-test  SalesLT  Product                          TABLE
data-science-test  SalesLT  ProductCategory                  TABLE
data-science-test  SalesLT  ProductDescription               TABLE
data-science-test  SalesLT  ProductModel                     TABLE
data-science-test  SalesLT  ProductModelProductDescription   TABLE
data-science-test  SalesLT  SalesOrderDetail                 TABLE
data-science-test  SalesLT  SalesOrderHeader                 TABLE
data-science-test  SalesLT  vGetAllCategories                VIEW
data-science-test  SalesLT  vProductAndDescription           VIEW
data-science-test  SalesLT  vProductModelCatalogDescript

In [123]:
def column_info(table, schema='SalesLT'):
    
    column_list = []
    indexes = (3,5,6)
    
    with conn.cursor() as cursor:
    
        for row in cursor.columns(table=table, schema=schema):
            column_data = [row[x] for x in indexes]
            column_list.append(column_data)
    
        headers = [cursor.description[x][0] for x in indexes]

    return column_list, headers


columns, headers = column_info('Address')
print(tabulate(columns, headers))

column_name    type_name           column_size
-------------  ----------------  -------------
AddressID      int identity                 10
AddressLine1   nvarchar                     60
AddressLine2   nvarchar                     60
City           nvarchar                     30
StateProvince  Name                         50
CountryRegion  Name                         50
PostalCode     nvarchar                     15
rowguid        uniqueidentifier             36
ModifiedDate   datetime                     23


In [150]:
import pprint

def constraint_info(table, schema='SalesLT'):
    
    with conn.cursor() as cursor:
        pk_list = cursor.primaryKeys(table=table, schema=schema).fetchall()
        pk_headers = [h[0] for h in cursor.description]
        fk_list = cursor.foreignKeys(table=table, schema=schema).fetchall()
        fk_headers = [h[0] for h in cursor.description]

    return pk_list, pk_headers, fk_list, fk_headers

pk_list, pk_headers, fk_list, fk_headers = constraint_info('Address')

print('Primary Keys\n============')
print(tabulate(pk_list, headers=pk_headers))
print()

print('Foreign Keys\n============')
print(tabulate(fk_list, headers=fk_headers))
print()


Primary Keys
table_cat          table_schem    table_name    column_name      key_seq  pk_name
-----------------  -------------  ------------  -------------  ---------  --------------------
data-science-test  SalesLT        Address       AddressID              1  PK_Address_AddressID

Foreign Keys
pktable_cat        pktable_schem    pktable_name    pkcolumn_name    fktable_cat        fktable_schem    fktable_name      fkcolumn_name      key_seq    update_rule    delete_rule  fk_name                                       pk_name                 deferrability
-----------------  ---------------  --------------  ---------------  -----------------  ---------------  ----------------  ---------------  ---------  -------------  -------------  --------------------------------------------  --------------------  ---------------
data-science-test  SalesLT          Address         AddressID        data-science-test  SalesLT          CustomerAddress   AddressID                1              1       

In [179]:
def constraint_info(table, schema='SalesLT'):
    pk_list = []
    fk_list = []
    pk_headers = []
    fk_headers = []
    pk_indexes = (3, )
    fk_indexes= (3, 6, 7)
    
    with conn.cursor() as cursor:
        
        for row in cursor.primaryKeys(table=table, schema=schema):
            data = [row[x] for x in pk_indexes]
            pk_list.append(data)
        pk_headers = [cursor.description[x][0] for x in pk_indexes]
        
        for row in cursor.foreignKeys(table=table, schema=schema):
            data = [row[x] for x in fk_indexes]
            fk_list.append(data)
        fk_headers = [cursor.description[x][0] for x in fk_indexes]

    return pk_list, pk_headers, fk_list, fk_headers

def display_constraints(table, schema='SalesLT'):
    
    pk_list, pk_headers, fk_list, fk_headers = constraint_info(table)
    
    output = ''
    
    output += f'TABLE = {schema}.{table}\n'
    output += '=========================\n'
    output += '\n'
    
    output += 'Primary Keys\n'
    output += tabulate(pk_list, headers=pk_headers) + '\n'
    output += '\n'

    output += 'Foreign Keys\n'
    output += tabulate(fk_list, headers=fk_headers)

    return output
    
table = 'Address'
print(display_constraints(table))




TABLE = SalesLT.Address

Primary Keys
column_name
-------------
AddressID

Foreign Keys
pkcolumn_name    fktable_name      fkcolumn_name
---------------  ----------------  ---------------
AddressID        CustomerAddress   AddressID
AddressID        SalesOrderHeader  BillToAddressID
AddressID        SalesOrderHeader  ShipToAddressID


In [166]:
test = ''
test.join('tttest')
print(test)

In [208]:
statement = """
SELECT sys.objects.name AS table_name
FROM sys.objects 
INNER JOIN sys.schemas ON sys.objects.schema_id = sys.schemas.schema_id
WHERE sys.schemas.name = 'SalesLT'
AND sys.objects.type = 'U' OR sys.objects.type = 'V'
"""

with conn.cursor() as cursor:
    cursor.execute(statement)
    for row in cursor.fetchall():
        print(*row)
    


database_firewall_rules
Customer
ProductModel
vProductModelCatalogDescription
ProductDescription
Product
ProductModelProductDescription
vProductAndDescription
ProductCategory
vGetAllCategories
Address
CustomerAddress
SalesOrderDetail
SalesOrderHeader


In [25]:
statement = """
SELECT 
    sys.columns.name as column_name, 
    sys.types.name as type, 
    sys.columns.max_length as max_length
FROM sys.objects
JOIN sys.columns 
    ON sys.objects.object_id = sys.columns.object_id
INNER JOIN sys.types
    ON sys.types.user_type_id = sys.columns.user_type_id
INNER JOIN sys.schemas 
    ON sys.objects.schema_id = sys.schemas.schema_id
WHERE sys.objects.name = 'CustomerAddress'
AND sys.schemas.name = 'SalesLT'
"""

with conn.cursor() as cursor:
    cursor.execute(statement)
    headers = [h[0] for h in cursor.description]
    results = cursor.fetchall()

print(tabulate(results, headers=headers))

column_name    type                max_length
-------------  ----------------  ------------
CustomerID     int                          4
AddressID      int                          4
rowguid        uniqueidentifier            16
ModifiedDate   datetime                     8


In [259]:
def columns_info(table, schema='SalesLT'):
    
    statement = f"""
    SELECT 
        sys.columns.name as column_name, 
        sys.types.name as type, 
        sys.columns.max_length as max_length
    FROM sys.objects
    JOIN sys.columns 
        ON sys.objects.object_id = sys.columns.object_id
    INNER JOIN sys.types
        ON sys.types.user_type_id = sys.columns.user_type_id
    INNER JOIN sys.schemas 
        ON sys.objects.schema_id = sys.schemas.schema_id
    WHERE sys.objects.name = '{table}'
    AND sys.schemas.name = '{schema}'
    """

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        results = cursor.fetchall()

    return results, headers

results, headers = columns_info('Address')
print(tabulate(results, headers=headers))

column_name    type                max_length
-------------  ----------------  ------------
AddressID      int                          4
AddressLine1   nvarchar                   120
AddressLine2   nvarchar                   120
City           nvarchar                    60
StateProvince  Name                       100
CountryRegion  Name                       100
PostalCode     nvarchar                    30
rowguid        uniqueidentifier            16
ModifiedDate   datetime                     8


In [65]:
def columns_info(table, schema='SalesLT'):
    
    statement = f"""
    SELECT 
        sys.columns.name as column_name, 
        sys.types.name as type, 
        sys.columns.max_length as max_length
    FROM sys.objects
    JOIN sys.columns 
        ON sys.objects.object_id = sys.columns.object_id
    INNER JOIN sys.types
        ON sys.types.user_type_id = sys.columns.user_type_id
    INNER JOIN sys.schemas 
        ON sys.objects.schema_id = sys.schemas.schema_id
    WHERE sys.objects.name = '{table}'
    AND sys.schemas.name = '{schema}'
    """

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        results = cursor.fetchall()

    return results, headers

results, headers = columns_info('Address')
print(tabulate(results, headers=headers))

column_name    type                max_length
-------------  ----------------  ------------
AddressID      int                          4
AddressLine1   nvarchar                   120
AddressLine2   nvarchar                   120
City           nvarchar                    60
PostalCode     nvarchar                    30
rowguid        uniqueidentifier            16
ModifiedDate   datetime                     8


In [24]:
def pk_info(table, schema='SalesLT'):
    
    statement = f"""
    SELECT
        sys.columns.name AS column_name,
        sys.indexes.name AS index_name,
        sys.columns.is_identity
    FROM sys.indexes
        inner join sys.index_columns  
            ON sys.indexes.object_id = sys.index_columns.object_id 
            AND sys.indexes.index_id = sys.index_columns.index_id
        inner join sys.columns 
            ON sys.index_columns.object_id = sys.columns.object_id 
            AND sys.columns.column_id = sys.index_columns.column_id
    WHERE sys.indexes.is_primary_key = 1
        and sys.indexes.object_ID = OBJECT_ID('{schema}.{table}');
    """

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        results = cursor.fetchall()

    return results, headers

results, headers = pk_info('CustomerAddress')
print(tabulate(results, headers=headers))

column_name    index_name                               is_identity
-------------  ---------------------------------------  -------------
CustomerID     PK_CustomerAddress_CustomerID_AddressID  False
AddressID      PK_CustomerAddress_CustomerID_AddressID  False


In [63]:
def fk_info(table, schema='SalesLT'):
    
    statement = f"""
    SELECT
        tab1.name AS table_name,
        col1.name AS column_name,
        tab2.name AS referenced_table,
        col2.name AS referenced_column
    FROM sys.foreign_key_columns fkc
    INNER JOIN sys.objects obj
        ON obj.object_id = fkc.constraint_object_id
    INNER JOIN sys.tables tab1
        ON tab1.object_id = fkc.parent_object_id
    INNER JOIN sys.schemas sch
        ON tab1.schema_id = sch.schema_id
    INNER JOIN sys.columns col1
        ON col1.column_id = parent_column_id AND col1.object_id = tab1.object_id
    INNER JOIN sys.tables tab2
        ON tab2.object_id = fkc.referenced_object_id
    INNER JOIN sys.columns col2
        ON col2.column_id = referenced_column_id AND col2.object_id = tab2.object_id
    WHERE sch.name = '{schema}' AND tab1.name = '{table}'
    """

    with conn.cursor() as cursor:
        cursor.execute(statement)
        headers = [h[0] for h in cursor.description]
        results = cursor.fetchall()

    return results, headers

results, headers = fk_info('CustomerAddress')
print(tabulate(results, headers=headers))

table_name       column_name    referenced_table    referenced_column
---------------  -------------  ------------------  -------------------
CustomerAddress  CustomerID     Customer            CustomerID
CustomerAddress  AddressID      Address             AddressID
